In [8]:
import re
import string
import pandas as pd
import numpy as np
from collections import Counter
from nltk.stem.porter import PorterStemmer

# N-Grams

In [17]:
with open('../Files/cien_soledad.txt', 'r') as file:
    corpus = ( 
        file.read()
        .replace('\n', '')
        .split(r'.')
    )
corpus_df = pd.DataFrame(corpus)
corpus_df.columns = ['sentences']
corpus_df['sentences'] = corpus_df.sentences.str.strip()
corpus_df['len'] = corpus_df.sentences.str

corpus_df = corpus_df[]

corpus_df.head()

SyntaxError: invalid syntax (<ipython-input-17-3a14d01b646a>, line 12)

## Encontrar las líneas que contengan números telefonicos

In [6]:
ans = tweets_df.copy()
phones = ans['text'].str.extractall(r"[^\w](\+\d{2})?(\s*\d+)+")
phones[1] = phones[1].str.strip()
phones = phones[phones[1].str.len() > 4]
phones = phones.fillna("")

phones

0             1
    match                   
177 0      +57    3223960100
239 0      +57       6655861
722 0             3208706120
    1      +47         69793
815 4           018000127400
877 1                  14000

In [7]:
(chr(0x1f000), chr(0x2f000))

('🀀', '\U0002f000')

In [8]:
test = "te @AuderH.Contáctanos 📞 al +57 3223960100 📞 estamos para complacer"
exp = re.compile(r'[🀀-𯀀]')
exp.search(test)

<re.Match object; span=(23, 24), match='📞'>

In [9]:

mask = tweets_df['text'].str.contains(r'[🀀-𯀀]')
ans = tweets_df[mask].head(10)
ans

,id_tweet,created_at,user_name,text
5,1047199536978911232,2018-10-02 14:00:00,Bogota,#NoOlvides 🧐 que las rutas alimentadoras 2-11 ...
10,1047287648782635008,2018-10-02 19:50:07,__Agnelli,@camilofacellif Después del accidente hago tod...
12,1047499932209430528,2018-10-03 09:53:40,melo18peralt,Todo el mundo hablando de amor y uno bien frío...
13,1047505442254016519,2018-10-03 10:15:33,DianaMorV,"@Venancioopinon Querido Venancio, gracias. 🦁❤️🇮🇩"
21,1047816604489269248,2018-10-04 06:52:00,MONTOYAANGELIK,#enlavialavida 🚴🏼‍♀️🚴🏼‍♂️#niunomas #Bocarejore...
23,1047920898479742977,2018-10-04 13:46:26,clajoesor,Me encanta!! Felicitaciones @VickyDavilaH me g...
42,1048446529848193025,2018-10-06 00:35:06,HSANCHEZ09,🔥🔥🇨🇴💎⚽🥇🏆🔥🔥 en La Concha FC https://t.co/LwUuR8...
48,1048612691538038784,2018-10-06 11:35:22,Brendamdelga,Buenos días 🤙🏼🤙🏼 https://t.co/vJrRQ9CjqV
59,1048911882113961984,2018-10-07 07:24:15,HLiinda,"""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""Ya Vendrán Mejo..."
64,1049024971589214209,2018-10-07 14:53:37,LauraNaoEsta,Como gane Bolsonaro en Brasil se acabó la educ...


# Tokenization

* Dividir tokens por espacios, “tab”, y “enters”
* Dividir por emoticones:
* NO divider hashtags
* Separar signos de puntuación del texto (tokesn individuales):
* Separar “Contractions” usando el “apostrophe”

In [88]:
def flat_list(arg):
    ans = []
    for li in arg:
        if isinstance(li, list):
            ans += li
        else:
            ans += [li]
    return ans

def tokenizer(text):
    separate_group = lambda match: "|" + match.group(0) + "|"

    space_re = re.compile (r'\s+')
    emoji_re = re.compile (r'([🀀-𯀀]+)')
    punctuation_re = re.compile (r"([\.,/\\:¿?\-$%()!¡]+)")
    contractions_re = re.compile (r"(n?'\w+)")

    token_list = space_re.split(text)

    token_list = [ emoji_re.sub(separate_group, txt) for txt in token_list ]
    token_list = [ txt.split('|') for txt in token_list ]
    token_list = flat_list(token_list)

    token_list = [ punctuation_re.sub(separate_group, txt) for txt in token_list ]
    token_list = [ txt.split('|') for txt in token_list ]
    token_list = flat_list(token_list)

    token_list = [ contractions_re.sub(separate_group, txt) for txt in token_list ]
    token_list = [ txt.split('|') for txt in token_list ]
    token_list = flat_list(token_list)

    token_list = [ txt for txt in token_list if len(txt) > 0 ]
    return token_list

In [90]:
text = "I'm at Centro Comercial @ParqueLaColina_ in Bogotá, Oliii😃😃😃 Bogota D.C. https://t.co/RQXSpmmifR"
tokens = tokenizer(text)
tokens

['I',
 "'m",
 'at',
 'Centro',
 'Comercial',
 '@ParqueLaColina_',
 'in',
 'Bogotá',
 ',',
 'Oliii',
 '😃😃😃',
 'Bogota',
 'D',
 '.',
 'C',
 '.',
 'https',
 '://',
 't',
 '.',
 'co',
 '/',
 'RQXSpmmifR']

## Normalizacion

In [121]:
#cargue Diccionario o Libro
def words(text):
    return re.findall(r'\w+', text.lower())
WORDS = Counter(words(open('../Files/cien_soledad.txt').read()))

def palabra_clave(word, N=sum(WORDS.values())): 
    return WORDS[word] / N

def corrector(word): 
    return max(candidatos(word), key=palabra_clave)

def candidatos(word): 
    return (cadena([word]) or cadena(busqueda(word)) or cadena(busquedad_val(word)) or [word])

def cadena(words): 
    return set(w for w in words if w in WORDS)

def busqueda(word):
    letters    = 'abcdefghijklmnopqrstuvwxyz'
    splits     = [(word[:i], word[i:])    for i in range(len(word) + 1)]
    deletes    = [L + R[1:]               for L, R in splits if R]
    transposes = [L + R[1] + R[0] + R[2:] for L, R in splits if len(R)>1]
    replaces   = [L + c + R[1:]           for L, R in splits if R for c in letters]
    inserts    = [L + c + R               for L, R in splits for c in letters]
    return set(deletes + transposes + replaces + inserts)

#se realiza busqueda por cada una de las paalbras
def busquedad_val(word): 
    return (e2 for e1 in busqueda(word) for e2 in busqueda(e1))

In [120]:
corrector('vow')

'voz'

##  Porter's Algorithm

In [108]:
stemmer = PorterStemmer()
(stemmer, text3)

(<PorterStemmer>, <Text: The Book of Genesis>)

In [107]:
[(stemmer.stem(word), word) for word in set(text3)]

r'),
 ('wing', 'winged'),
 ('breaketh', 'breaketh'),
 ('ophir', 'Ophir'),
 ('are', 'Areli'),
 ('maiden', 'maiden'),
 ('hil', 'hil'),
 ('gether', 'Gether'),
 ('aw', 'aw'),
 ('wonder', 'wondering'),
 ('judgment', 'judgment'),
 ('thou', 'Thou'),
 ('nourish', 'nourish'),
 ('ey', 'ey'),
 ('knoweth', 'knoweth'),
 ('tree', 'tree'),
 ('camel', 'camels'),
 ('seven', 'sevens'),
 ('shame', 'shamed'),
 ('My', 'My'),
 ('shed', 'shed'),
 ('establish', 'establish'),
 ('G', 'G'),
 ('bethel', 'Bethel'),
 ('wrap', 'wrapped'),
 ('slew', 'slew'),
 ('been', 'been'),
 ('again', 'again'),
 ('meant', 'meant'),
 ('maidserv', 'maidservants'),
 ('activ', 'activity'),
 ('peradventur', 'peradventure'),
 ('exchang', 'exchange'),
 ('widowhood', 'widowhood'),
 ('shammah', 'Shammah'),
 ('honour', 'honourable'),
 ('bear', 'bearing'),
 ('s', 's'),
 ('purchas', 'purchased'),
 ('rephaim', 'Rephaims'),
 ('anoth', 'another'),
 ('withhold', 'withhold'),
 ('pit', 'pit'),
 ('husbandman', 'husbandman'),
 ('hul', 'Hul'),
 ('inte